In [1]:
import cv2
import numpy as np
import threading
import time
import timeit

# Function to handle YOLO object detection
def detect_objects(frame, net, output_layers, classes, confidence_threshold=0.5, nms_threshold=0.4):
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    detected_objects = []

    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence_text = f"{round(confidences[i] * 100, 2)}%"
            detected_objects.append((label, confidence_text, (x, y, w, h)))

    return detected_objects

# Function to handle face detection (Haar Cascade)
def detect_faces_haar(gray_frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Function to handle face detection (DNN)
def detect_faces_dnn(frame, net):
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300), mean=(104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    faces = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, x_max, y_max) = box.astype("int")
            faces.append((x, y, x_max - x, y_max - y))

    return faces

# Function to evaluate performance of algorithms
def evaluate_algorithms():
    # Initialize webcam
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise Exception("Webcam not accessible")

    # Load models for evaluation
    yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = yolo_net.getLayerNames()
    yolo_output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

    dnn_face_net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    haar_times = []
    dnn_face_times = []
    yolo_times = []

    haar_accuracy = []
    dnn_face_accuracy = []
    yolo_accuracy = []

    # Loop for performance testing
    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Haar Cascade Detection
        start = timeit.default_timer()
        haar_faces = detect_faces_haar(gray_frame)
        haar_times.append(timeit.default_timer() - start)
        haar_accuracy.append(len(haar_faces))

        # DNN Face Detection
        start = timeit.default_timer()
        dnn_faces = detect_faces_dnn(frame, dnn_face_net)
        dnn_face_times.append(timeit.default_timer() - start)
        dnn_face_accuracy.append(len(dnn_faces))

        # YOLO Object Detection
        start = timeit.default_timer()
        yolo_objects = detect_objects(frame, yolo_net, yolo_output_layers, classes)
        yolo_times.append(timeit.default_timer() - start)
        yolo_accuracy.append(len(yolo_objects))

    cap.release()

    print("Performance Metrics:")
    print(f"Haar Cascade - Average Time: {np.mean(haar_times):.4f} seconds, Average Detections: {np.mean(haar_accuracy):.2f}")
    print(f"DNN Face Detection - Average Time: {np.mean(dnn_face_times):.4f} seconds, Average Detections: {np.mean(dnn_face_accuracy):.2f}")
    print(f"YOLO Object Detection - Average Time: {np.mean(yolo_times):.4f} seconds, Average Detections: {np.mean(yolo_accuracy):.2f}")

    fastest = min((np.mean(haar_times), "Haar Cascade"), (np.mean(dnn_face_times), "DNN Face"), (np.mean(yolo_times), "YOLO"), key=lambda x: x[0])
    most_detections = max((np.mean(haar_accuracy), "Haar Cascade"), (np.mean(dnn_face_accuracy), "DNN Face"), (np.mean(yolo_accuracy), "YOLO"), key=lambda x: x[0])

    print(f"Fastest Algorithm: {fastest[1]} with {fastest[0]:.4f} seconds")
    print(f"Highest Average Detections: {most_detections[1]} with {most_detections[0]:.2f} detections")

if __name__ == "__main__":
    try:
        evaluate_algorithms()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cv2.destroyAllWindows()


Performance Metrics:
Haar Cascade - Average Time: 0.0471 seconds, Average Detections: 1.00
DNN Face Detection - Average Time: 0.0239 seconds, Average Detections: 1.00
YOLO Object Detection - Average Time: 0.3385 seconds, Average Detections: 1.00
Fastest Algorithm: DNN Face with 0.0239 seconds
Highest Average Detections: Haar Cascade with 1.00 detections


In [3]:
import cv2
import numpy as np
import threading
import time
import timeit

# Function to handle YOLO object detection
def detect_objects(frame, net, output_layers, classes, confidence_threshold=0.5, nms_threshold=0.4):
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > confidence_threshold:
                center_x = int(detection[0] * frame.shape[1])
                center_y = int(detection[1] * frame.shape[0])
                w = int(detection[2] * frame.shape[1])
                h = int(detection[3] * frame.shape[0])

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence_threshold, nms_threshold)
    detected_objects = []

    if len(indices) > 0:
        for i in indices.flatten():
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence_text = f"{round(confidences[i] * 100, 2)}%"
            detected_objects.append((label, confidence_text, (x, y, w, h)))

    return detected_objects

# Function to handle face detection (Haar Cascade)
def detect_faces_haar(gray_frame):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return faces

# Function to handle face detection (DNN)
def detect_faces_dnn(frame, net):
    blob = cv2.dnn.blobFromImage(frame, scalefactor=1.0, size=(300, 300), mean=(104.0, 177.0, 123.0))
    net.setInput(blob)
    detections = net.forward()
    faces = []

    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
            (x, y, x_max, y_max) = box.astype("int")
            faces.append((x, y, x_max - x, y_max - y))

    return faces

# Function to handle face detection (MTCNN)
def detect_faces_mtcnn(frame, mtcnn_detector):
    detections = mtcnn_detector.detect_faces(frame)
    faces = []

    for detection in detections:
        box = detection['box']
        x, y, w, h = box
        faces.append((x, y, w, h))

    return faces

# Enhanced webcam handling
def initialize_webcam():
    cap = None
    for i in range(5):
        cap = cv2.VideoCapture(i)
        if cap.isOpened():
            print(f"Webcam initialized using index {i}.")
            return cap
    raise Exception("Webcam not accessible. Please check the connection or permissions.")

# Function to evaluate performance of algorithms
def evaluate_algorithms():
    # Initialize webcam
    cap = initialize_webcam()

    # Load models for evaluation
    yolo_net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    layer_names = yolo_net.getLayerNames()
    yolo_output_layers = [layer_names[i - 1] for i in yolo_net.getUnconnectedOutLayers()]

    dnn_face_net = cv2.dnn.readNetFromCaffe("deploy.prototxt", "res10_300x300_ssd_iter_140000.caffemodel")

    from mtcnn.mtcnn import MTCNN
    mtcnn_detector = MTCNN()

    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]

    metrics = {
        'haar': {'times': [], 'detections': []},
        'dnn_face': {'times': [], 'detections': []},
        'mtcnn': {'times': [], 'detections': []},
        'yolo': {'times': [], 'detections': []}
    }

    for i in range(10):
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Haar Cascade Detection
        start = timeit.default_timer()
        haar_faces = detect_faces_haar(gray_frame)
        metrics['haar']['times'].append(timeit.default_timer() - start)
        metrics['haar']['detections'].append(len(haar_faces))

        # DNN Face Detection
        start = timeit.default_timer()
        dnn_faces = detect_faces_dnn(frame, dnn_face_net)
        metrics['dnn_face']['times'].append(timeit.default_timer() - start)
        metrics['dnn_face']['detections'].append(len(dnn_faces))

        # MTCNN Face Detection
        start = timeit.default_timer()
        mtcnn_faces = detect_faces_mtcnn(frame, mtcnn_detector)
        metrics['mtcnn']['times'].append(timeit.default_timer() - start)
        metrics['mtcnn']['detections'].append(len(mtcnn_faces))

        # YOLO Object Detection
        start = timeit.default_timer()
        yolo_objects = detect_objects(frame, yolo_net, yolo_output_layers, classes)
        metrics['yolo']['times'].append(timeit.default_timer() - start)
        metrics['yolo']['detections'].append(len(yolo_objects))

    cap.release()

    print("Performance Metrics:")
    for algo, data in metrics.items():
        avg_time = np.mean(data['times'])
        avg_detections = np.mean(data['detections'])
        print(f"{algo.upper()} - Average Time: {avg_time:.4f} seconds, Average Detections: {avg_detections:.2f}")

    fastest = min((np.mean(metrics['haar']['times']), "Haar Cascade"),
                  (np.mean(metrics['dnn_face']['times']), "DNN Face"),
                  (np.mean(metrics['mtcnn']['times']), "MTCNN"),
                  (np.mean(metrics['yolo']['times']), "YOLO"),
                  key=lambda x: x[0])

    print(f"Fastest Algorithm: {fastest[1]} with {fastest[0]:.4f} seconds")

if __name__ == "__main__":
    try:
        evaluate_algorithms()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cv2.destroyAllWindows()


Webcam initialized using index 0.


c:\Users\dell\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


Performance Metrics:
HAAR - Average Time: 2.3783 seconds, Average Detections: 0.10
DNN_FACE - Average Time: 1.7461 seconds, Average Detections: 0.20
MTCNN - Average Time: 2.0227 seconds, Average Detections: 0.00
YOLO - Average Time: 9.4820 seconds, Average Detections: 1.60
Fastest Algorithm: DNN Face with 1.7461 seconds
